In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Problem 1 Verion 1

In [112]:
m = AbstractModel()

m.FOOD_TYPES = Set()
m.MACROS = Set()

m.amount = Var(m.FOOD_TYPES, domain = NonNegativeReals)
m.CostPerPound = Param(m.FOOD_TYPES)

m.FoodMacros = Param(m.FOOD_TYPES, m.MACROS)

m.MinimumMacros = Param(m.MACROS)

def cost_of_food(m):
    return sum_product(m.amount, m.CostPerPound, index = m.FOOD_TYPES)

m.total_cost = Objective(rule = cost_of_food, sense = minimize)

def minimum_macro_requirement(model, m):
    return quicksum(model.amount[t] * model.FoodMacros[t, m] for t in model.FOOD_TYPES) >= model.MinimumMacros[m]

m.minimum_macro_requirements = Constraint(m.MACROS, rule = minimum_macro_requirement)

In [113]:
instanceData = { None: {
    'FOOD_TYPES': {None: ['A', 'B', 'C']},
    'MACROS': {None: ['Protein', 'Carbs']},
    'CostPerPound': {'A': 5, 'B': 15, 'C': 20},
    'FoodMacros': {('A', 'Protein'): 4, ('A', 'Carbs'): 15, ('B', 'Protein'): 8, ('B', 'Carbs'): 11, ('C', 'Protein'): 8, ('C', 'Carbs'): 0},
    'MinimumMacros': {'Protein': 20, 'Carbs': 10}
}}

In [114]:
instance = m.create_instance(instanceData)

In [115]:
opt = SolverFactory('glpk')
opt.solve(instance, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpe3jtft71.glpk.raw --wglp /tmp/tmp95cx7ryc.glpk.glp --cpxlp
 /tmp/tmpjedt_2sd.pyomo.lp
Reading problem data from '/tmp/tmpjedt_2sd.pyomo.lp'...
2 rows, 3 columns, 5 non-zeros
26 lines were read
Writing problem data to '/tmp/tmp95cx7ryc.glpk.glp'...
18 lines were written
GLPK Simplex Optimizer 5.0
2 rows, 3 columns, 5 non-zeros
Preprocessing...
2 rows, 3 columns, 5 non-zeros
Scaling...
 A: min|aij| =  4.000e+00  max|aij| =  1.500e+01  ratio =  3.750e+00
GM: min|aij| =  7.782e-01  max|aij| =  1.285e+00  ratio =  1.651e+00
EQ: min|aij| =  6.055e-01  max|aij| =  1.000e+00  ratio =  1.651e+00
Constructing initial basis...
Size of triangular part is 2
      0: obj =   0.000000000e+00 inf =   3.357e+00 (2)
      2: obj =   4.666666667e+01 inf =   0.000e+00 (0)
*     3: obj =   2.500000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing 

{'Problem': [{'Name': 'unknown', 'Lower bound': 25.0, 'Upper bound': 25.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 3, 'Number of nonzeros': 5, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.0069315433502197266}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [116]:
instance.display()

Model unknown

  Variables:
    amount : Size=3, Index=FOOD_TYPES
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   5.0 :  None : False : False : NonNegativeReals
          B :     0 :   0.0 :  None : False : False : NonNegativeReals
          C :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    total_cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    minimum_macro_requirements : Size=2
        Key     : Lower : Body : Upper
          Carbs :  10.0 : 75.0 :  None
        Protein :  20.0 : 20.0 :  None


## Version 2

In [117]:
!ls

/bin/bash: //home/shaunak/.miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
'Diet Problem.ipynb'		 'Pyomo Experimentation.ipynb'
 DP_input_food.csv		  Q1.ipynb
 DP_input_food_nutrients.csv	  School_Reopening.ipynb
 DP_input_min_intake.csv	  SRO_input_grades.csv
'Network Optimization.ipynb'	  SRO_input_rooms.csv
'Production Optimization.ipynb'  'Use Case 1 - Room specification.ipynb'


In [118]:
nutrients_df = pd.read_csv("DP_input_food_nutrients.csv")
min_intake_df = pd.read_csv("DP_input_min_intake.csv")
food_df = pd.read_csv("DP_input_food.csv")

In [119]:
min_intake_df

,Nutr,Req_grams
0,Calories,1800
1,Protein,45
2,Fat,40
3,Sat.Fat,0
4,Fiber,26
5,Carbs,202


In [120]:
food_df

,Food,Cost_dollars
0,Beef,6.00
1,Cheese,0.50
2,Corn meal,0.50
3,Cornflakes,0.50
4,Cows milk,3.28
5,Crackers,0.50
6,Cream cheese,0.30
7,Eggs raw,0.40
8,Flounder,5.00
9,Flour,1.00


In [121]:
instanceData = { None: {
    'FOOD_TYPES': {None: food_df["Food"].unique()},
    'MACROS': {None: nutrients_df["Nutr"].unique()},
    'CostPerPound': food_df.set_index(["Food"]).to_dict()["Cost_dollars"],
    'FoodMacros': nutrients_df.set_index(["Food", "Nutr"]).to_dict()["Grams"],
    'MinimumMacros': min_intake_df.set_index(["Nutr"]).to_dict()["Req_grams"]
}}

In [122]:
instance = m.create_instance(instanceData)

In [123]:
opt.solve(instance, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpazop7n88.glpk.raw --wglp /tmp/tmp6bf8kidu.glpk.glp --cpxlp
 /tmp/tmp0mp7ovps.pyomo.lp
Reading problem data from '/tmp/tmp0mp7ovps.pyomo.lp'...
6 rows, 21 columns, 91 non-zeros
160 lines were read
Writing problem data to '/tmp/tmp6bf8kidu.glpk.glp'...
148 lines were written
GLPK Simplex Optimizer 5.0
6 rows, 21 columns, 91 non-zeros
Preprocessing...
5 rows, 21 columns, 78 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  7.480e+02  ratio =  7.480e+02
GM: min|aij| =  2.051e-01  max|aij| =  4.875e+00  ratio =  2.376e+01
EQ: min|aij| =  4.208e-02  max|aij| =  1.000e+00  ratio =  2.376e+01
Constructing initial basis...
Size of triangular part is 5
      0: obj =   0.000000000e+00 inf =   1.977e+01 (5)
      5: obj =   4.192714797e+00 inf =   0.000e+00 (0)
*     7: obj =   3.967211055e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (52263 bytes)


{'Problem': [{'Name': 'unknown', 'Lower bound': 3.96721105527638, 'Upper bound': 3.96721105527638, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 21, 'Number of nonzeros': 91, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.005259513854980469}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [124]:
instance.display()

Model unknown

  Variables:
    amount : Size=21, Index=FOOD_TYPES
        Key           : Lower : Value            : Upper : Fixed : Stale : Domain
                 Beef :     0 :              0.0 :  None : False : False : NonNegativeReals
               Cheese :     0 :              0.0 :  None : False : False : NonNegativeReals
              Chicken :     0 :              0.0 :  None : False : False : NonNegativeReals
            Corn meal :     0 :              0.0 :  None : False : False : NonNegativeReals
           Cornflakes :     0 :              0.0 :  None : False : False : NonNegativeReals
            Cows milk :     0 :              0.0 :  None : False : False : NonNegativeReals
             Crackers :     0 :              0.0 :  None : False : False : NonNegativeReals
         Cream cheese :     0 : 1.23115577889447 :  None : False : False : NonNegativeReals
             Eggs raw :     0 :              0.0 :  None : False : False : NonNegativeReals
             Flounder :